In [3]:
!pip install prettytable


Defaulting to user installation because normal site-packages is not writeable


In [1]:
# Insert Data

from pymongo import MongoClient, InsertOne

# Koneksi ke database
client = MongoClient("mongodb://localhost:27017/")
db = client["university_db"]
courses_collection = db["courses"]
students_collection = db["students"]

# Hapus data lama agar tidak duplikat
courses_collection.delete_many({})
students_collection.delete_many({})

# Dataset baru untuk courses
courses_data = [
    {
        "course": "Math 101",
        "department": "Mathematics",
        "enrollments": 32,
        "students": ["Alice", "Bob", "Charlie", "David"]
    },
    {
        "course": "Math 202",
        "department": "Mathematics",
        "enrollments": 18,
        "students": ["Eve", "Frank"]
    },
    {
        "course": "CS 102",
        "department": "Computer Science",
        "enrollments": 27,
        "students": ["Grace", "Heidi", "Ivan"]
    },
    {
        "course": "CS 205",
        "department": "Computer Science",
        "enrollments": 41,
        "students": ["Judy", "Kevin", "Liam", "Mia", "Nina"]
    },
    {
        "course": "CS 303",
        "department": "Computer Science",
        "enrollments": 23,
        "students": ["Oscar", "Peggy"]
    },
    {
        "course": "History 201",
        "department": "History",
        "enrollments": 15,
        "students": ["Quinn", "Rita"]
    },
    {
        "course": "History 310",
        "department": "History",
        "enrollments": 21,
        "students": ["Sam", "Tina", "Uma"]
    },
    {
        "course": "Physics 202",
        "department": "Physics",
        "enrollments": 17,
        "students": ["Victor", "Wendy"]
    },
    {
        "course": "Physics 303",
        "department": "Physics",
        "enrollments": 29,
        "students": ["Xander", "Yara", "Zane"]
    },
    {
        "course": "Economics 101",
        "department": "Economics",
        "enrollments": 25,
        "students": ["Aaron", "Bella", "Cody"]
    },
    {
        "course": "Economics 205",
        "department": "Economics",
        "enrollments": 19,
        "students": ["Dina", "Ethan"]
    },
    {
        "course": "Biology 110",
        "department": "Biology",
        "enrollments": 34,
        "students": ["Fiona", "George", "Hannah", "Isaac"]
    }
]

# Insert dataset ke koleksi
courses_collection.insert_many(courses_data)

# Dataset untuk students (disesuaikan agar bisa di-lookup nanti)
students_data = [
    {"name": "Alice", "major": "Mathematics"},
    {"name": "Bob", "major": "Mathematics"},
    {"name": "Charlie", "major": "Mathematics"},
    {"name": "David", "major": "Mathematics"},
    {"name": "Grace", "major": "Computer Science"},
    {"name": "Heidi", "major": "Computer Science"},
    {"name": "Ivan", "major": "Computer Science"},
    {"name": "Judy", "major": "Computer Science"},
    {"name": "Kevin", "major": "Computer Science"},
    {"name": "Liam", "major": "Computer Science"},
    {"name": "Mia", "major": "Computer Science"},
    {"name": "Nina", "major": "Computer Science"},
    {"name": "Oscar", "major": "Computer Science"},
    {"name": "Peggy", "major": "Computer Science"},
    {"name": "Quinn", "major": "History"},
    {"name": "Rita", "major": "History"},
    {"name": "Sam", "major": "History"},
    {"name": "Tina", "major": "History"},
    {"name": "Uma", "major": "History"},
    {"name": "Victor", "major": "Physics"},
    {"name": "Wendy", "major": "Physics"},
    {"name": "Xander", "major": "Physics"},
    {"name": "Yara", "major": "Physics"},
    {"name": "Zane", "major": "Physics"},
    {"name": "Aaron", "major": "Economics"},
    {"name": "Bella", "major": "Economics"},
    {"name": "Cody", "major": "Economics"},
    {"name": "Dina", "major": "Economics"},
    {"name": "Ethan", "major": "Economics"},
    {"name": "Fiona", "major": "Biology"},
    {"name": "George", "major": "Biology"},
    {"name": "Hannah", "major": "Biology"},
    {"name": "Isaac", "major": "Biology"}
]

students_collection.insert_many(students_data)

print("Dataset baru berhasil dimasukkan ke database.")


Dataset baru berhasil dimasukkan ke database.


In [5]:
# Homework 1

from pymongo import MongoClient
from prettytable import PrettyTable
import json

# Koneksi ke database
client = MongoClient("mongodb://localhost:27017/")
db = client["university_db"]
courses_collection = db["courses"]

print("\n" + "="*75)
print("COUNT OF COURSES PER DEPARTMENT".center(75))
print("="*75)

# Pipeline agregasi
pipeline = [
    {'$group': {'_id': '$department', 'course_count': {'$sum': 1}}},
    {'$sort': {'_id': 1}}
]
results = list(courses_collection.aggregate(pipeline))

# Tampilkan Table format
table = PrettyTable(['Department', 'Number of Courses'])
for r in results:
    table.add_row([r['_id'], r['course_count']])
print(table)



                      COUNT OF COURSES PER DEPARTMENT                      
+------------------+-------------------+
|    Department    | Number of Courses |
+------------------+-------------------+
|     Biology      |         1         |
| Computer Science |         3         |
|    Economics     |         2         |
|     History      |         2         |
|   Mathematics    |         2         |
|     Physics      |         2         |
+------------------+-------------------+


In [6]:
# Homework 2

from pymongo import MongoClient
from prettytable import PrettyTable

client = MongoClient("mongodb://localhost:27017/")
db = client["university_db"]
courses_collection = db["courses"]

print("\n" + "="*75)
print("COURSES WITH ENROLLMENTS > 25 IN COMPUTER SCIENCE".center(75))
print("="*75)

# Pipeline agregasi
pipeline = [
    {'$match': {'department': 'Computer Science', 'enrollments': {'$gt': 25}}},
    {'$project': {'_id': 0, 'course': 1, 'department': 1, 'enrollments': 1}},
    {'$sort': {'enrollments': -1}}
]
results = list(courses_collection.aggregate(pipeline))

# Tampilkan hasil dalam tabel
table = PrettyTable(['Course', 'Department', 'Enrollments'])
for r in results:
    table.add_row([r['course'], r['department'], r['enrollments']])

print(table)



             COURSES WITH ENROLLMENTS > 25 IN COMPUTER SCIENCE             
+--------+------------------+-------------+
| Course |    Department    | Enrollments |
+--------+------------------+-------------+
| CS 205 | Computer Science |      41     |
| CS 102 | Computer Science |      27     |
+--------+------------------+-------------+


In [9]:
# Homework 3

from pymongo import MongoClient
from prettytable import PrettyTable
import json

client = MongoClient("mongodb://localhost:27017/")
db = client["university_db"]
courses_collection = db["courses"]

print("\n" + "="*85)
print("JOIN COURSES WITH STUDENTS".center(75))
print("="*85)

# Pipeline lookup
pipeline = [
    {
        '$lookup': {
            'from': 'students',
            'localField': 'students',
            'foreignField': 'name',
            'as': 'student_info'
        }
    },
    {
        '$project': {
            '_id': 0,
            'course': 1,
            'department': 1,
            'enrollments': 1,
            'student_info.name': 1
        }
    }
]

results = list(courses_collection.aggregate(pipeline))

# Table Format
table = PrettyTable(['Course', 'Department', 'Enrollments', 'Students'])
for r in results:
    students = ", ".join([s['name'] for s in r.get('student_info', [])])
    table.add_row([r['course'], r['department'], r['enrollments'], students])

print(table)



                         JOIN COURSES WITH STUDENTS                        
+---------------+------------------+-------------+------------------------------+
|     Course    |    Department    | Enrollments |           Students           |
+---------------+------------------+-------------+------------------------------+
|    Math 101   |   Mathematics    |      32     |  Alice, Bob, Charlie, David  |
|    Math 202   |   Mathematics    |      18     |                              |
|     CS 102    | Computer Science |      27     |      Grace, Heidi, Ivan      |
|     CS 205    | Computer Science |      41     | Judy, Kevin, Liam, Mia, Nina |
|     CS 303    | Computer Science |      23     |         Oscar, Peggy         |
|  History 201  |     History      |      15     |         Quinn, Rita          |
|  History 310  |     History      |      21     |        Sam, Tina, Uma        |
|  Physics 202  |     Physics      |      17     |        Victor, Wendy         |
|  Physics 303  |    